#   The 8-9pm Rangebreakout Stoporder Method 
---

##  Flow


![Test 1 Result](https://github.com/algorembrant/Pinescript.TradingView-Indicators.and.Strategies/blob/main/Images/Screenshot%202025-11-15%20135957.png?raw=true)


##  About

| Property             | Details                                    |
|---------------------|-------------------------------------------|
| **Ticker/Instrument** | XAUUSD                                     |
| **Broker**            | OANDA (TradingView only)                  |
| **Timeframe**         | Non-specific                               |
| **Timezone**          | Asia/Manila (UTC+8)                        |
| **Session**           | New York (customed 1h)                    |
| **Method**            | Range Breakout                             |

##  Details

Locate the x-y coordinate of candles formed from 8pm to 9pm (20 oclock in asianmanila timeframe).

Among these candles, find the highest point and lowest point, and make vertical lines until it makes a rectangle from those vertices. This will serve as range.
   
    ┌──────────────  sessionHigh
    │
    │   ████  ← 8PM–9PM candles
    │
    └──────────────  sessionLow

Locate the y-coordinate and mark the high, low, and midpoit of the range.

    ┌────────────── HIGH   (green)
    │   ████
    │   ████
     ────────────── MID    (orange)
    │   ████
    │   ████
    └────────────── LOW    (red)

The high will serve as buystop and to calculate/locate it's TP y-coordinate, absolute value of high minus mid and times it by two, then add it to the high(buystop price) to project the y-coordinate of it's TP. 

Inverse the logic or mirror it for the sellstop.

The midpoint will serve as SL for both orders.

**One range one setup two orders four outcomes** 

    ─────────────── TP
    
    
    
     
     
    ┌─────────────► buystop
    │    ████    │
    │    ████    │
     ────────────── SL
    │    ████    │
    │    ████    │
    └─────────────► sellstop
    
       
    
    
    
    ─────────────── TP

locate and highlight the candle at 4am. This will serve as the validation for the setup.

In [ ]:
//Pinescript Indicator
//@version=5
indicator("8PM–9PM Range Strategy (Midpoint as SL) + 4AM Candle", overlay=true, max_lines_count=500, max_labels_count=500, max_boxes_count=200)

// SETTINGS
tz = "Asia/Manila"
startHour = 20
endHour   = 21

// TIME DETECTION
barHour = hour(time, tz)
barMin  = minute(time, tz)
inSession    = barHour >= startHour and barHour < endHour
sessionOpen  = barHour == startHour and barMin == 0
sessionClose = barHour == endHour and barMin == 0

// TRACK RANGE DURING SESSION
var float sHigh = na
var float sLow  = na
var int   sStart = na
var int   sEnd   = na
var box   sBox = na

if sessionOpen
    sHigh := high
    sLow  := low
    sStart := bar_index
    sEnd := na

if inSession
    sHigh := math.max(sHigh, high)
    sLow  := math.min(sLow, low)

if sessionClose and not na(sHigh)
    sEnd := bar_index

    // RANGE BOX
    if not na(sBox)
        box.delete(sBox)
    sBox := box.new(
         left = sStart,
         right = sEnd,
         top = sHigh,
         bottom = sLow,
         bgcolor = color.new(#000000, 85),
         border_color = color.new(#000000, 0))

    // CORE LEVELS
    mid = (sHigh + sLow) / 2

    // BUY & SELL SETUPS
    buyStop = sHigh
    buySL   = mid
    buyTP   = sHigh + (math.abs(sHigh - mid) * 2)

    sellStop = sLow
    sellSL   = mid
    sellTP   = sLow - (math.abs(mid - sLow) * 2)

    // DRAW RANGE LINES
    line.new(sStart, sHigh, sEnd, sHigh, color=color.new(color.green, 0), style=line.style_dotted)
    line.new(sStart, sLow,  sEnd, sLow,  color=color.new(color.red, 0),   style=line.style_dotted)
    line.new(sStart, mid,   sEnd, mid,   color=color.new(color.orange, 0), style=line.style_dotted)

    // DRAW BUY SETUP
    line.new(sEnd, buyStop, sEnd + 5, buyStop, color=color.new(color.lime, 0), width=2)
    line.new(sEnd, buySL,   sEnd + 5, buySL,   color=color.new(color.orange, 0), width=2, style=line.style_dashed)
    line.new(sEnd, buyTP,   sEnd + 5, buyTP,   color=color.new(color.lime, 0), width=2, style=line.style_dotted)

    label.new(sEnd + 5, buyStop, text=str.format("BUY STOP\n{0}", str.tostring(buyStop, format.mintick)), style=label.style_label_left, color=color.new(color.lime, 0), textcolor=color.white, size=size.tiny)
    label.new(sEnd + 5, buySL,   text=str.format("SL (MID)\n{0}",   str.tostring(buySL,   format.mintick)), style=label.style_label_left, color=color.new(color.orange, 0), textcolor=color.white, size=size.tiny)
    label.new(sEnd + 5, buyTP,   text=str.format("BUY TP\n{0}",     str.tostring(buyTP,   format.mintick)), style=label.style_label_left, color=color.new(color.lime, 0), textcolor=color.white, size=size.tiny)

    // DRAW SELL SETUP
    line.new(sEnd, sellStop, sEnd + 5, sellStop, color=color.new(color.red, 0), width=2)
    line.new(sEnd, sellSL,   sEnd + 5, sellSL,   color=color.new(color.orange, 0), width=2, style=line.style_dashed)
    line.new(sEnd, sellTP,   sEnd + 5, sellTP,   color=color.new(color.red, 0), width=2, style=line.style_dotted)

    label.new(sEnd + 5, sellStop, text=str.format("SELL STOP\n{0}", str.tostring(sellStop, format.mintick)), style=label.style_label_left, color=color.new(color.red, 0), textcolor=color.white, size=size.tiny)
    label.new(sEnd + 5, sellSL,   text=str.format("SL (MID)\n{0}",  str.tostring(sellSL,   format.mintick)), style=label.style_label_left, color=color.new(color.orange, 0), textcolor=color.white, size=size.tiny)
    label.new(sEnd + 5, sellTP,   text=str.format("SELL TP\n{0}",   str.tostring(sellTP,   format.mintick)), style=label.style_label_left, color=color.new(color.red, 0), textcolor=color.white, size=size.tiny)

// 4AM CANDLE HIGHLIGHT
is4am = barHour == 4 and barMin == 0
var box box4am = na
if is4am
    if not na(box4am)
        box.delete(box4am)
    box4am := box.new(left = bar_index, right = bar_index + 1, top = high, bottom = low, bgcolor = color.new(color.blue, 75), border_color = color.new(color.blue, 0))
    label.new(bar_index, high, "4 AM", style=label.style_label_down, color=color.new(color.blue, 0), textcolor=color.white, size=size.tiny)


##  **Setup Overview:**  
This framework defines the conditions for executing the trading strategy.

![Template](https://github.com/algorembrant/Pinescript.TradingView-Indicators.and.Strategies/blob/main/Images/Screenshot%202025-11-15%20153904.png?raw=true)

---

## Rules

1. Use **Asia/Manila timezone (UTC+8)**.  
2. Identify the **8 PM – 9 PM price range**.  
3. Candles outside this range will guide trading decisions.  
4. The developing price (including wicks) **must revisit the mid-price** of the range before proceeding.  
5. Once the mid-price is revisited:  
   - Place a **Buy Stop** at the range **high**.  
   - Place a **Sell Stop** at the range **low**.  
6. **Take Profit (TP):** 2R multiple from the entry.  
7. **Stop Loss (SL):** mid-price for both Buy Stop and Sell Stop.  
8. Both Buy Stop and Sell Stop **may trigger** in the same setup, valid until the **4 AM candle**.  
9. Maximum outcomes per setup: **2 wins, 2 losses, or a mix**.  
10. If the trade hasn’t hit any target by 4 AM, **cancel the orders**.  

## Risk Management

- Starting capital: **$2,000**  
- Risk per trade: **3% of current capital** (applies separately to Buy Stop and Sell Stop).  

In [ ]:
//Pinescript Strategy
//@version=5
strategy("8PM–9PM Range Strategy + 4AM Candle", overlay=true, initial_capital=2000)

// TIME SETTINGS
tz = "Asia/Manila"
startHour = 20
endHour   = 21
riskPerc = 3.0

barHour = hour(time, tz)
barMin  = minute(time, tz)
inSession = barHour >= startHour and barHour < endHour
sessionOpen = barHour == startHour and barMin == 0
sessionClose = barHour == endHour and barMin == 0
is4am = barHour == 4 and barMin == 0

// RANGE VARIABLES (from indicator)
var float sHigh = na
var float sLow  = na
var int sStart = na
var int sEnd   = na
var bool touchedMid = false
var bool ordersPlaced = false

if sessionOpen
    sHigh := high
    sLow  := low
    sStart := bar_index
    sEnd := na
    touchedMid := false
    ordersPlaced := false

if inSession
    sHigh := math.max(sHigh, high)
    sLow  := math.min(sLow, low)

if sessionClose and not na(sHigh)
    sEnd := bar_index

// USE INDICATOR COORDINATES
mid = (sHigh + sLow)/2
buyStop  = sHigh
buySL    = mid
buyTP    = sHigh + 2*(sHigh - mid)
sellStop = sLow
sellSL   = mid
sellTP   = sLow - 2*(mid - sLow)

// DETECT MID TOUCH (entry condition)
if not na(mid) and not ordersPlaced
    if high >= mid and low <= mid
        touchedMid := true

// PLACE ORDERS BASED ON INDICATOR COORDINATES
if touchedMid and not ordersPlaced
    riskAmount = strategy.equity * riskPerc / 100.0

    stopDistBuy = math.abs(buyStop - buySL)
    stopDistSell = math.abs(sellStop - sellSL)
    qtyBuy = stopDistBuy > 0 ? math.max(1, math.round(riskAmount / stopDistBuy)) : 0
    qtySell = stopDistSell > 0 ? math.max(1, math.round(riskAmount / stopDistSell)) : 0

    if qtyBuy > 0
        strategy.entry("BUY_STOP", strategy.long, qty=qtyBuy, stop=buyStop)
        strategy.exit("EXIT_BUY", from_entry="BUY_STOP", stop=buySL, limit=buyTP)

    if qtySell > 0
        strategy.entry("SELL_STOP", strategy.short, qty=qtySell, stop=sellStop)
        strategy.exit("EXIT_SELL", from_entry="SELL_STOP", stop=sellSL, limit=sellTP)

    ordersPlaced := true

// CANCEL / CLOSE ORDERS AT 4AM
if is4am
    strategy.cancel("BUY_STOP")
    strategy.cancel("SELL_STOP")
    if strategy.position_size > 0
        strategy.close("BUY_STOP")
    if strategy.position_size < 0
        strategy.close("SELL_STOP")
    touchedMid := false
    ordersPlaced := false


## Implementation Note

Use the **y-coordinates from the indicator** to calculate the range, entries, SL, and TP targets.

![Strategy Example](https://github.com/algorembrant/Pinescript.TradingView-Indicators.and.Strategies/blob/main/Images/Screenshot%202025-11-15%20131419.png?raw=true)

---

##  Result

![Test 1 Result](https://github.com/algorembrant/Pinescript.TradingView-Indicators.and.Strategies/blob/main/Images/Screenshot%202025-11-15%20132923.png?raw=true)

## Test 1: Notice 

| Topic / Flaws       | Details                                                                                                             | Status       |
|--------------------|---------------------------------------------------------------------------------------------------------------------|-------------|
| **Position sizing** | The bot does not fully account for P&L when sizing positions. For example, it risks 3% per trade, but the loss can exceed this and gains may exceed 6%. | Unresolved  |
| **R-multiple**      | Trades indicate the possibility of reaching a 4R multiple upon review.                                             | Unresolved  |
| **Win rate**        | The current win rate is 40–43%, which is lower than ideal.                                                          | Unresolved  |

This strategy is naturally profitable, but I aim to **increase the R-multiples and win rate** through additional testing.

---

## Adjustments

For getting higher 4R-multiple instead of 2R.
Same range, same TP price but different SL size.
Stoploss are is tighter.

stoploss calculation:

$ buystop.stoploss   =   (ABS(buystop.entryprice - sellstop.entryprice)/4)*1 - buystop.entryprice $

$ sellstop.stoploss   =   (ABS(sellstop.entryprice - buystop.entryprice)/4)*1 + sellstop.entryprice $

takeprofit calculation:

$ buystop.takeprofit   =   (ABS(buystop.entryprice - sellstop.entryprice)/4)*4 + buystop.entryprice $

$ sellstop.takeprofit   =   (ABS(sellstop.entryprice - buystop.entryprice)/4)*4 - sellstop.entryprice $



##  Pinescript

In [ ]:
//@version=5
indicator("8PM–9PM Range Strategy (Tighter SL for 4R) + 4AM Candle", overlay=true, max_lines_count=500, max_labels_count=500, max_boxes_count=200)

// SETTINGS
tz = "Asia/Manila"
startHour = 20
endHour   = 21

// TIME DETECTION
barHour = hour(time, tz)
barMin  = minute(time, tz)
inSession    = barHour >= startHour and barHour < endHour
sessionOpen  = barHour == startHour and barMin == 0
sessionClose = barHour == endHour and barMin == 0

// TRACK RANGE DURING SESSION
var float sHigh = na
var float sLow  = na
var int   sStart = na
var int   sEnd   = na
var box   sBox = na

if sessionOpen
    sHigh := high
    sLow  := low
    sStart := bar_index
    sEnd := na

if inSession
    sHigh := math.max(sHigh, high)
    sLow  := math.min(sLow, low)

if sessionClose and not na(sHigh)
    sEnd := bar_index

    // RANGE BOX
    if not na(sBox)
        box.delete(sBox)
    sBox := box.new(
         left = sStart,
         right = sEnd,
         top = sHigh,
         bottom = sLow,
         bgcolor = color.new(#000000, 85),
         border_color = color.new(#000000, 0))

    // CORE LEVELS
    buyStop  = sHigh
    sellStop = sLow

    // use 'rng' instead of reserved word 'range'
    rng   = math.abs(buyStop - sellStop)
    quarter = rng / 4

    // NEW STOPLOSS CALC (TIGHTER)
    buySL  = buyStop  - quarter
    sellSL = sellStop + quarter

    // NEW TAKEPROFIT CALC (4R)
    buyTP  = buyStop  + rng
    sellTP = sellStop - rng

    // Mid is kept for drawing reference
    mid = (sHigh + sLow) / 2

    // DRAW RANGE LINES
    line.new(sStart, sHigh, sEnd, sHigh, color=color.new(color.green, 0), style=line.style_dotted)
    line.new(sStart, sLow,  sEnd, sLow,  color=color.new(color.red, 0),   style=line.style_dotted)
    line.new(sStart, mid,   sEnd, mid,   color=color.new(color.orange, 0), style=line.style_dotted)

    // DRAW BUY SETUP
    line.new(sEnd, buyStop, sEnd + 5, buyStop, color=color.new(color.lime, 0), width=2)
    line.new(sEnd, buySL,   sEnd + 5, buySL,   color=color.new(color.orange, 0), width=2, style=line.style_dashed)
    line.new(sEnd, buyTP,   sEnd + 5, buyTP,   color=color.new(color.lime, 0), width=2, style=line.style_dotted)

    label.new(sEnd + 5, buyStop, text=str.format("BUY STOP\n{0}", str.tostring(buyStop, format.mintick)), style=label.style_label_left, color=color.new(color.lime, 0), textcolor=color.white, size=size.tiny)
    label.new(sEnd + 5, buySL,   text=str.format("SL (Quarter)\n{0}", str.tostring(buySL, format.mintick)), style=label.style_label_left, color=color.new(color.orange, 0), textcolor=color.white, size=size.tiny)
    label.new(sEnd + 5, buyTP,   text=str.format("BUY TP\n{0}",     str.tostring(buyTP, format.mintick)), style=label.style_label_left, color=color.new(color.lime, 0), textcolor=color.white, size=size.tiny)

    // DRAW SELL SETUP
    line.new(sEnd, sellStop, sEnd + 5, sellStop, color=color.new(color.red, 0), width=2)
    line.new(sEnd, sellSL,   sEnd + 5, sellSL,   color=color.new(color.orange, 0), width=2, style=line.style_dashed)
    line.new(sEnd, sellTP,   sEnd + 5, sellTP,   color=color.new(color.red, 0), width=2, style=line.style_dotted)

    label.new(sEnd + 5, sellStop, text=str.format("SELL STOP\n{0}", str.tostring(sellStop, format.mintick)), style=label.style_label_left, color=color.new(color.red, 0), textcolor=color.white, size=size.tiny)
    label.new(sEnd + 5, sellSL,   text=str.format("SL (Quarter)\n{0}",  str.tostring(sellSL,   format.mintick)), style=label.style_label_left, color=color.new(color.orange, 0), textcolor=color.white, size=size.tiny)
    label.new(sEnd + 5, sellTP,   text=str.format("SELL TP\n{0}",   str.tostring(sellTP,   format.mintick)), style=label.style_label_left, color=color.new(color.red, 0), textcolor=color.white, size=size.tiny)

// 4AM CANDLE HIGHLIGHT
is4am = barHour == 4 and barMin == 0
var box box4am = na
if is4am
    if not na(box4am)
        box.delete(box4am)
    box4am := box.new(left = bar_index, right = bar_index + 1, top = high, bottom = low, bgcolor = color.new(color.blue, 75), border_color = color.new(color.blue, 0))
    label.new(bar_index, high, "4 AM", style=label.style_label_down, color=color.new(color.blue, 0), textcolor=color.white, size=size.tiny)


##  Setup

![image](https://github.com/algorembrant/Pinescript.TradingView-Indicators.and.Strategies/blob/main/Images/Screenshot%202025-11-15%20160104.png?raw=true)

## Rules

1. Use **Asia/Manila timezone (UTC+8)**.  
2. Identify the **8 PM – 9 PM price range**.  
3. Candles outside this range will guide trading decisions.  
4. The developing price (including wicks) **must revisit the mid-price** of the range before proceeding.  
5. Once the mid-price is revisited:  
   - Place a **Buy Stop** at the range **high**.  
   - Place a **Sell Stop** at the range **low**.  
6. **Take Profit (TP):** 4R multiple from the entry.  
7. **Stop Loss (SL):** quarter-price for both respective Buy Stop and Sell Stop.  
8. Both Buy Stop and Sell Stop **may trigger** in the same setup, valid until the **4 AM candle**.  
9. Maximum outcomes per setup: **2 wins, 2 losses, or a mix**.  
10. If the trade hasn’t hit any target by 4 AM, **cancel the orders**.  

## Concept:

Starting capital: $2000

Risk per trade: 3% of current capital (applies independently to Buy Stop and Sell Stop)

Stop loss: quarter of the 8–9 PM range (SL = quarter)

Calculate position size per trade so that risk in USD = 3% of account

$  Position Size (units)   =  (Account Risk in USD / SL in price units) $
	​


##  Pinescript Strategy

In [ ]:
//@version=5
strategy("8PM–9PM Range Strategy + 4AM + Risk Mgmt", overlay=true,
  initial_capital=2000, 
  default_qty_type=strategy.fixed, 
  default_qty_value=1, 
  max_bars_back=500)

// ========== INPUTS ==========
tz = "Asia/Manila"
startHour = input.int(20, "Range Start Hour (24h)", minval=0, maxval=23)
endHour   = input.int(21, "Range End Hour (24h)", minval=0, maxval=23)
riskPerc  = input.float(3.0, "Risk % per trade", step=0.1)

// ========== TIME ==========
barHour = hour(time, tz)
barMin  = minute(time, tz)
inSession    = barHour >= startHour and barHour < endHour
sessionOpen  = barHour == startHour and barMin == 0
sessionClose = barHour == endHour and barMin == 0
is4am = barHour == 4 and barMin == 0

// ========== PERSISTENT RANGE VARIABLES ==========
var float sHigh = na
var float sLow  = na
var int   sStart = na
var int   sEnd   = na
var box   sBox = na

// ========== DECLARE PRICE LEVEL VARIABLES ==========
var float buyStop = na
var float sellStop = na
var float rng = na
var float quarter = na
var float buySL = na
var float sellSL = na
var float buyTP = na
var float sellTP = na
var float mid = na

// ========== SESSION RANGE TRACKING ==========
if sessionOpen
    sHigh := high
    sLow  := low
    sStart := bar_index
    sEnd := na

if inSession and not na(sStart)
    sHigh := math.max(sHigh, high)
    sLow  := math.min(sLow, low)

if sessionClose and not na(sHigh)
    sEnd := bar_index

    if not na(sBox)
        box.delete(sBox)
    sBox := box.new(left = sStart, right = sEnd, top = sHigh, bottom = sLow, bgcolor = color.new(color.black, 85), border_color = color.new(color.black, 0))

    // reset session flags
    var bool setup_active = false
    var bool revisited_mid = false
    var bool orders_sent = false
    setup_active := true
    revisited_mid := false
    orders_sent := false

// Persistent flags
var bool _setup_active = false
var bool _revisited_mid = false
var bool _orders_sent = false

if sessionClose and not na(sHigh)
    _setup_active := true
    _revisited_mid := false
    _orders_sent := false

if na(sHigh)
    _setup_active := false

// ========== CALCULATE PRICE LEVELS ==========
if not na(sHigh)
    buyStop  := sHigh
    sellStop := sLow
    rng := math.abs(buyStop - sellStop)
    quarter := rng / 4
    buySL := buyStop - quarter
    sellSL := sellStop + quarter
    buyTP := buyStop + rng
    sellTP := sellStop - rng
    mid := (sHigh + sLow) / 2

// Draw range lines
if sessionClose and not na(sHigh)
    line.new(sStart, sHigh, sEnd, sHigh, color=color.new(color.green, 0), style=line.style_dotted)
    line.new(sStart, sLow,  sEnd, sLow,  color=color.new(color.red, 0),   style=line.style_dotted)
    line.new(sStart, mid,   sEnd, mid,   color=color.new(color.orange, 0), style=line.style_dotted)

// ========== MID REVISIT ==========
if _setup_active and not _revisited_mid and not na(mid)
    if (high >= mid) and (low <= mid)
        _revisited_mid := true

// ========== RISK MANAGEMENT: POSITION SIZE ==========
calc_qty(sl_price) =>
    // risk in USD
    acct_risk = strategy.equity * riskPerc / 100
    // position size in units = account risk / SL distance
    qty = acct_risk / math.abs(sl_price - close)
    qty

// ========== PLACE STOP ORDERS ==========
if _setup_active and _revisited_mid and not _orders_sent and not na(buyStop)
    buy_qty  = calc_qty(buySL)
    sell_qty = calc_qty(sellSL)

    strategy.entry("BuyStop", strategy.long, stop=buyStop, qty=buy_qty)
    strategy.exit("BuyExit", from_entry="BuyStop", stop=buySL, limit=buyTP)

    strategy.entry("SellStop", strategy.short, stop=sellStop, qty=sell_qty)
    strategy.exit("SellExit", from_entry="SellStop", stop=sellSL, limit=sellTP)

    _orders_sent := true

// ========== CANCEL ORDERS AT 4AM ==========
if is4am
    strategy.cancel("BuyStop")
    strategy.cancel("SellStop")
    _setup_active := false
    _revisited_mid := false
    _orders_sent := false

// ========== 4AM HIGHLIGHT ==========
var box box4am = na
if is4am
    if not na(box4am)
        box.delete(box4am)
    box4am := box.new(left = bar_index, right = bar_index + 1, top = high, bottom = low,
                       bgcolor = color.new(color.blue, 75), border_color = color.new(color.blue, 0))
    label.new(bar_index, high, "4 AM", style=label.style_label_down,
              color=color.new(color.blue, 0), textcolor=color.white, size=size.tiny)


##  Test 2 Results

![image](https://github.com/algorembrant/Pinescript.TradingView-Indicators.and.Strategies/blob/main/Images/Screenshot%202025-11-15%20162112.png?raw=true)

##  Test 2 Results
![image](https://github.com/algorembrant/Pinescript.TradingView-Indicators.and.Strategies/blob/main/Images/Screenshot%202025-11-15%20162726.png?raw=true)

## Conclusion

Its not the best to tighten the SL area.

Its not the best to put a 4am cut off limit

#   If we remove the 4am cutoff Logic

If we remove the 4 AM cutoff logic, the strategy will no longer cancel pending orders at 4 AM — orders will stay active until they hit SL or TP. Everything else (mid-price revisit, Buy/Sell Stop, 4R TP, quarter SL, risk management) stays intact.

In [ ]:
//@version=5
strategy("8PM–9PM Range Strategy + Risk Mgmt (No 4AM cutoff)", overlay=true,
  initial_capital=2000, 
  default_qty_type=strategy.fixed, 
  default_qty_value=1, 
  max_bars_back=500)

// ========== INPUTS ==========
tz = "Asia/Manila"
startHour = input.int(20, "Range Start Hour (24h)", minval=0, maxval=23)
endHour   = input.int(21, "Range End Hour (24h)", minval=0, maxval=23)
riskPerc  = input.float(3.0, "Risk % per trade", step=0.1)

// ========== TIME ==========
barHour = hour(time, tz)
barMin  = minute(time, tz)
inSession    = barHour >= startHour and barHour < endHour
sessionOpen  = barHour == startHour and barMin == 0
sessionClose = barHour == endHour and barMin == 0

// ========== PERSISTENT RANGE VARIABLES ==========
var float sHigh = na
var float sLow  = na
var int   sStart = na
var int   sEnd   = na
var box   sBox = na

// ========== DECLARE PRICE LEVEL VARIABLES ==========
var float buyStop = na
var float sellStop = na
var float rng = na
var float quarter = na
var float buySL = na
var float sellSL = na
var float buyTP = na
var float sellTP = na
var float mid = na

// ========== SESSION RANGE TRACKING ==========
if sessionOpen
    sHigh := high
    sLow  := low
    sStart := bar_index
    sEnd := na

if inSession and not na(sStart)
    sHigh := math.max(sHigh, high)
    sLow  := math.min(sLow, low)

if sessionClose and not na(sHigh)
    sEnd := bar_index

    if not na(sBox)
        box.delete(sBox)
    sBox := box.new(left = sStart, right = sEnd, top = sHigh, bottom = sLow,bgcolor = color.new(color.black, 85), border_color = color.new(color.black, 0))

    // reset session flags
    var bool setup_active = false
    var bool revisited_mid = false
    var bool orders_sent = false
    setup_active := true
    revisited_mid := false
    orders_sent := false

// Persistent flags
var bool _setup_active = false
var bool _revisited_mid = false
var bool _orders_sent = false

if sessionClose and not na(sHigh)
    _setup_active := true
    _revisited_mid := false
    _orders_sent := false

if na(sHigh)
    _setup_active := false

// ========== CALCULATE PRICE LEVELS ==========
if not na(sHigh)
    buyStop  := sHigh
    sellStop := sLow
    rng := math.abs(buyStop - sellStop)
    quarter := rng / 4
    buySL := buyStop - quarter
    sellSL := sellStop + quarter
    buyTP := buyStop + rng
    sellTP := sellStop - rng
    mid := (sHigh + sLow) / 2

// Draw range lines
if sessionClose and not na(sHigh)
    line.new(sStart, sHigh, sEnd, sHigh, color=color.new(color.green, 0), style=line.style_dotted)
    line.new(sStart, sLow,  sEnd, sLow,  color=color.new(color.red, 0),   style=line.style_dotted)
    line.new(sStart, mid,   sEnd, mid,   color=color.new(color.orange, 0), style=line.style_dotted)

// ========== MID REVISIT ==========
if _setup_active and not _revisited_mid and not na(mid)
    if (high >= mid) and (low <= mid)
        _revisited_mid := true

// ========== RISK MANAGEMENT: POSITION SIZE ==========
calc_qty(sl_price) =>
    acct_risk = strategy.equity * riskPerc / 100
    distance = math.abs(sl_price - close)
    distance := distance < syminfo.mintick ? syminfo.mintick : distance
    qty = acct_risk / distance
    qty := math.max(qty, 1)   // minimum 1 unit
    qty

// ========== PLACE STOP ORDERS ==========
if _setup_active and _revisited_mid and not _orders_sent and not na(buyStop)
    buy_qty  = calc_qty(buySL)
    sell_qty = calc_qty(sellSL)

    strategy.entry("BuyStop", strategy.long, stop=buyStop, qty=buy_qty)
    strategy.exit("BuyExit", from_entry="BuyStop", stop=buySL, limit=buyTP)

    strategy.entry("SellStop", strategy.short, stop=sellStop, qty=sell_qty)
    strategy.exit("SellExit", from_entry="SellStop", stop=sellSL, limit=sellTP)

    _orders_sent := true


#   Test 3 Results

![image](https://github.com/algorembrant/Pinescript.TradingView-Indicators.and.Strategies/blob/main/Images/Screenshot%202025-11-15%20164303.png?raw=true)

at this point, i am conviced that i should work on the riskmanagement and position sizing logic first.

# 📈 Position Lot Size Formula for Risk Management

This formula helps traders determine the appropriate size of a trading position based on their risk tolerance, capital, and trade setup.

## Formula

$$
\text{position_lotsize} = \frac{(\text{risk%_in_decimal}) \cdot \text{Capital}}{|\text{Entry_price} - \text{Stoploss_price}|} \times 0.01\_\text{standard}
$$

##  Explanation

- **risk%_in_decimal**: The percentage of capital you're willing to risk, expressed as a decimal (e.g., 2% → 0.02).
- **Capital**: Your total trading capital.
- **Entry_price**: The price at which you enter the trade.
- **Stoploss_price**: The price at which you exit the trade to limit losses.
- **0.01_standard**: A scaling factor, likely referring to standard lot size (e.g., 1 standard lot = 100,000 units).

## Use Case

This formula ensures that each trade risks only a fixed percentage of your capital, helping maintain consistent risk management and avoid oversized positions.

---

## Formula 

$	buy = buystop, buylimit, buymarket	$

$	buy_stoploss_price =(entry_price - ticks)	$

$	buy_takeprofit_price =( entry_price + ticks)	$

$	sell = sellstop, selllimit, sellmarket	$

$	sell_stoploss_price =( entry_price + ticks)	$

$	sell_takeprofit_price =( entry_price - ticks)	$


The `buy` is a type of order.The `buy_stoploss_price` is the stop loss for buy order.The `buy_takeprofit_price` is the take profit for buy order.The `sell` is a type of order.The `sell_stoploss_price` is the stop loss for sell order.The `sell_takeprofit_price` is the take profit for sell order.

# 📊 Trading Order Price Calculations

This table summarizes how to calculate stop loss and take profit prices for different types of buy and sell orders.

| Order Type | Entry Type                         | Stop Loss Formula                     | Take Profit Formula                   |
|------------|------------------------------------|---------------------------------------|---------------------------------------|
| Buy        | `buystop`, `buylimit`, `buymarket` | `buy_stoploss_price = entry_price - ticks` | `buy_takeprofit_price = entry_price + ticks` |
| Sell       | `sellstop`, `selllimit`, `sellmarket` | `sell_stoploss_price = entry_price + ticks` | `sell_takeprofit_price = entry_price - ticks` |

## 🧾 Notes

- **Buy Orders**: Expect price to rise. Stop loss is below entry; take profit is above.
- **Sell Orders**: Expect price to fall. Stop loss is above entry; take profit is below.
- **ticks**: Represents the price movement unit used to define risk and reward thresholds.

Use this table to quickly reference how to set your stop loss and take profit levels based on order direction.


